<a href="https://colab.research.google.com/github/jordanburdett/IrisflowerDetection/blob/master/irisDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

01 Prove : Assignment - Classifier Intro

In [1]:
import pandas as pd

url = "https://byui-cs.github.io/cs450-course/week01/iris.data"
data = pd.read_csv(url)

print(data)

     sepal_length  sepal_width  petal_length  petal_width         species
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

[150 rows x 5 columns]


Split the data into two arrays features: measurements & Targets: species names

In [0]:
import numpy as np
targets = data['species'].to_numpy()
features = data[['sepal_length', 'sepal_width',
                'petal_length', 'petal_width']].to_numpy()


randomly split the data using SKlearn train_test_split method

In [0]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(features, targets, test_size=.3)

Create a KnnClassifier



In [4]:
from collections import Counter

class KnnClassifier:
  def fit(self, data, targets):
    self.data = data
    self.targets = targets
  
  #####
  # Calculates the distance between two points
  def calcDistance(self, x, y):
    return np.linalg.norm(x-y)

  ####
  # Finds the nearest neighbor based on target data
  def findNearestNeighbor(self, data, k):
    # Compare data with all of my stored self.data and the the 'k' amount of nearest neighbors
    # Create an array of distances from data
    distances = []
    for trainData in self.data:
      distances.append(self.calcDistance(trainData, data))

    # Find the index's of the 'k' amount of closest members
    sorted_indexes = np.argsort(distances)[:k]

    # Create nearest neighbors array
    nearestNeighbors = []
    for index in sorted_indexes:
      nearestNeighbors.append(self.targets[index])

    # Count to see which one is the most common and return
    counts = Counter(nearestNeighbors)

    return counts.most_common(1)[0][0]
    
  def predict(self, data, k):
    self.kNearest = k
    self.testData = data
    predictionArray = []
    # Find the nearest neighbor for each Set of data
    for currentData in self.testData:
      predictionArray.append(self.findNearestNeighbor(currentData, k))

    return predictionArray


print(test_data[:2])
print(test_targets[:2])

[[7.7 2.8 6.7 2. ]
 [6.  2.7 5.1 1.6]]
['Iris-virginica' 'Iris-versicolor']


Test the Classifier


In [5]:
classifier = KnnClassifier()
classifier.fit(train_data, train_targets)

predictions = classifier.predict(test_data, 3)
predictions

numCorrect = 0

for i in range(len(predictions)):
  if predictions[i] == test_targets[i]:
    numCorrect += 1

accuracy = (numCorrect / len(predictions)) * 100

print(accuracy)

del predictions, accuracy, numCorrect

97.77777777777777


Compare to Off the shelve implementation!


In [6]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(train_data, train_targets)
predictions = classifier.predict(test_data)

numCorrect = 0

for i in range(len(predictions)):
  if predictions[i] == test_targets[i]:
    numCorrect += 1

accuracy = (numCorrect / len(predictions)) * 100

print(accuracy)

97.77777777777777


Not sure why But I found some Covid-19 Data that im going to run through the classifier... Doesn't make a whole lot of sense to do this but I thought it would be good practice in getting a larger dataset loaded in here! And to be honest I thought it'd be fun!

In [7]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
data = pd.read_csv(url)

data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,tests_units
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14294,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14295,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14296,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14297,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
targets = data['location'].to_numpy()
features = data[['total_cases', 'new_cases',
                'total_deaths', 'new_deaths']].to_numpy()

train_data, test_data, train_targets, test_targets = train_test_split(features, targets, test_size=.3)

print(train_data)



[[655   0  20   0]
 [  0   0   0   0]
 [  0   0   0   0]
 ...
 [126   2   3   0]
 [  3   0   1   0]
 [174  30   2   0]]


In [0]:
classifier = KnnClassifier()
classifier.fit(train_data, train_targets)

predictions = classifier.predict(test_data, 3)


In [22]:
numCorrect = 0

for i in range(len(predictions)):
  if predictions[i] == test_targets[i]:
    numCorrect += 1

accuracy = (numCorrect / len(predictions)) * 100

print(len(predictions))
print(numCorrect)
print(accuracy)

4290
486
11.328671328671328


As you can see... This is not good at predicting what country based on amount infected... But logically it really shouldn’t So I'm counting this as a win. It is cool that I can take this same Knn Classifier and run it with different kinds of data and it still works. With my implementation it took about 8 minutes to run 

Now to try it with the off the shelve implementation to see if it runs faster

In [24]:
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(train_data, train_targets)
predictions = classifier.predict(test_data)

numCorrect = 0

for i in range(len(predictions)):
  if predictions[i] == test_targets[i]:
    numCorrect += 1

accuracy = (numCorrect / len(predictions)) * 100

print(accuracy)

11.375291375291376


Basically Instant. I assume this is because the calculations for distance are being done with a compiled C library. It could also do some things to not have to calculate all of the distances even when it is super far away.... But im not sure.. Super interesting though.